Download template MNI-HCP which Tian atlas is defined in from templateflow

In [ ]:

from templateflow import api
api.get('MNI152NLin6Asym', resolution=1, suffix='T1w')

Plot tian atlas in different scales

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
from subcortex_visualization.plotting import plot_subcortical_data

for s in ["S1", "S2"]:
    plot_subcortical_data(atlas=f"Melbourne_{s}", hemisphere='both', cmap="hsv")
    fig = plt.gcf()
    ax = plt.gca()
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')         
    plt.show()

Visualizing connectome matrix

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
from cmap import Colormap
from matplotlib.colors import LinearSegmentedColormap
from nilearn.plotting import plot_matrix
import pandas as pd
import numpy as np

In [27]:
df = pd.read_csv("/Users/payamsadeghishabestari/temp_folder/dti/asjt/conn/tian_S3_conn.csv", header=None)
cm = Colormap('colorbrewer:OrRd_9')
combined_cmap = LinearSegmentedColormap.from_list("", cm(np.linspace(0, 1, 500)), N=1000)

labels_fname = "./data/Tian2020MSA/3T/Subcortex-Only/Tian_Subcortex_S3_3T_label.txt"
labels = np.loadtxt(labels_fname, dtype=str)

fig, ax = plt.subplots(figsize=(10, 8))
plot_matrix(df, labels=labels, cmap=combined_cmap, axes=ax, colorbar=True, vmax=0.9, vmin=0)

for label in ax.get_xticklabels()[:25]:
    label.set_color('lightgreen')
for label in ax.get_yticklabels()[:25]:
    label.set_color('lightgreen')

for label in ax.get_xticklabels()[25:]:
    label.set_color('lavender')
for label in ax.get_yticklabels()[25:]:
    label.set_color('lavender')

fig.patch.set_facecolor('black')
ax.set_facecolor('black')          

# Make grid lines white for visibility (optional)
for spine in ax.spines.values():
    spine.set_color('white')


plt.show()